<a href="https://colab.research.google.com/github/Keval-Trivedi299/CSRBOX_IBMSKILLBUILD/blob/main/Industry_Innovation_and_Infrastructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [1]:
!pip install -qU \
  google-generativeai==0.8.5 \
  google-ai-generativelanguage==0.6.15 \
  langgraph \
  langchain \
  langchain-google-genai \
  openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.0 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# B. Secure Gemini API Key input

In [3]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your gemini key")

Enter your gemini key··········


# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2)

# STEP 3: Node to ask user for symptom

In [5]:
def get_problem(state: dict) -> dict:
  problem = input("This is an Expert AI guide for your problem , Please enter the situation to be solved")
  state["problem"] = problem
  return state

# STEP 4: Node to classify the symptom

In [6]:
def classify_problem(state : dict ) -> dict:
  prompt = (
      "You are a helpful Infrastructure Guardian assistant. Classify the user's concern below into one of the categories:\n"
        "- Awareness\n- Action\n- Emergency\n\n"
        f"Problem: {state['problem']}\n"
        "Respond with only one word: Awareness, Action, or Emergency.\n"
        "#Example: input: Our rural roads are in terrible condition and block access to markets. → output: Action"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM Classifies the problem as : {category}") #debug
  state["category"]=category

# STEP 5: Router logic to route to the correct node


In [7]:
def problem_router(state: dict) -> str:
    cat = state["category"].lower()
    if "awareness" in cat:
        return "awareness"
    elif "emergency" in cat:
        return "emergency"
    elif "action" in cat:
        return "action"
    else:
        return "awareness"


# STEP 6: Category-specific response nodes


In [8]:
def awareness_node(state: dict) -> dict:
    prompt = (
        f"You are a tech and infrastructure educator.\n"
        f"User's issue: {state['problem']}\n"
        "Give a detailed but clear explanation to raise awareness about this problem related to SDG 9."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def emergency_node(state: dict) -> dict:
    prompt = (
        f"You are a critical infrastructure response advisor.\n"
        f"Situation: {state['problem']}\n"
        "Provide urgent guidance or steps needed to resolve the infrastructure-related emergency quickly and safely."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def action_node(state: dict) -> dict:
    prompt = (
        f"You are an SDG 9 expert focused on real-world solutions.\n"
        f"User's concern: {state['problem']}\n"
        "Suggest specific, actionable steps the user can take to address this industry, innovation, or infrastructure issue."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state


# STEP 7: Build LangGraph


In [9]:
builder = StateGraph(dict)

builder.set_entry_point("get_problem")

builder.add_node("get_problem", get_problem)
builder.add_node("classify", classify_problem)
builder.add_node("awareness", awareness_node)
builder.add_node("emergency", emergency_node)
builder.add_node("action", action_node)

builder.add_edge("get_problem", "classify")

builder.add_conditional_edges("classify", problem_router, {
    "awareness": "awareness",
    "emergency": "emergency",
    "action": "action"
})

builder.add_edge("awareness", END)
builder.add_edge("emergency", END)
builder.add_edge("action", END)


# STEP 8: Compile and invoke the graph


In [14]:
graph = builder.compile()
final_state = graph.invoke({})
print("Final Output:")
print(final_state["answer"])

This is an Expert AI guide for your problem , Please enter the situation to be solvedtraffic lights is not on
LLM Classifies the problem as : Emergency
Final Output:
## Urgent Guidance: Traffic Signal Outage

**Priority 1: Immediate Safety Measures (First Responders & On-Scene Personnel)**

1. **Isolate the Area:**  Immediately dispatch law enforcement to the affected intersection(s).  Their priority is to establish a safe perimeter and direct traffic manually.  Use caution tape, flares, and/or emergency vehicles to delineate the affected area and warn approaching drivers.  This is paramount to prevent accidents.

2. **Assess the Situation:**  Determine the extent of the outage.  Is it a single intersection, a wider area, or a localized power failure?  Identify any visible damage to the traffic signals or power lines.  Report any downed power lines immediately to the appropriate utility company – **DO NOT APPROACH DOWNED POWER LINES.**

3. **Activate Emergency Response Plan:**  Contact